In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

from scipy.io import loadmat
from sklearn.metrics import mean_squared_error, mean_absolute_error
import scipy.stats as stats
from tqdm.auto import tqdm

import torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 4. Load stronger YOLOv5 model (yolov5m instead of nano)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)
model.to(device)
model.eval()

model.conf = 0.12
model.iou = 0.60
model.classes = [0]
model.max_det = 300

print("YOLOv5m model loaded and configured for crowded scenes.")

def count_persons_yolov5(image_path):
    """
    Returns the number of 'person' detections in the image
    using the configured YOLOv5m model.
    """
    with torch.no_grad():

        results = model(image_path, size=960, augment=True)

    det = results.xyxy[0]
    return int(det.shape[0])


Using device: cpu


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-12-9 Python-3.12.12 torch-2.9.0+cu126 CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


YOLOv5m model loaded and configured for crowded scenes.


In [ ]:
# Quick visual check on one frame

sample_row = df.iloc[500]
sample_path = os.path.join(FRAMES_DIR, sample_row.image_name)

print("Sample image:", sample_path)
print("GT count:", sample_row['count'])

# The model object already has conf, iou, classes, and max_det set from cell Uz4oaWAaUch7.
# We pass the image path as the first argument and optionally specify size and augment.
results = model(
    sample_path, # image path as positional argument
    size=960,
    augment=True
)

annotated_imgs = results.render()
img_bgr = annotated_imgs[0] # passing only one image to the model
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(7,5))
plt.imshow(img_rgb)
plt.title("YOLOv5m person detections")
plt.axis("off")
plt.show()

print("YOLO predicted count:", count_persons_yolov5(sample_path))

Sample image: /content/drive/MyDrive/archive/frames (1)/seq_000501.jpg
GT count: 26


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/archive/frames (1)/seq_000501.jpg'

In [ ]:
print(f"Checking directory: {FRAMES_DIR}")
if os.path.exists(FRAMES_DIR):
    print("Directory exists.")
    files = os.listdir(FRAMES_DIR)
    print(f"Number of files in directory: {len(files)}")
    if files:
        print("First 5 files:")
        for f in files[:5]:
            print(f)
    else:
        print("Directory is empty.")
else:
    print("Directory does NOT exist. Please ensure the 'frames (1)' folder is in your Google Drive at the specified path.")

In [ ]:
# SILENT YOLO EVALUATION CELL

import warnings
warnings.filterwarnings("ignore")   # Hide all warnings

from contextlib import redirect_stdout
import io

# N_FRAMES_EVAL = len(df)
N_FRAMES_EVAL = 800   # uncomment for faster testing

eval_df = df.iloc[:N_FRAMES_EVAL].copy()

gt_counts = []
pred_counts = []

# Suppress YOLO internal prints
f = io.StringIO()
with redirect_stdout(f):
    for _, row in tqdm(eval_df.iterrows(), total=len(eval_df)):
        img_path = os.path.join(FRAMES_DIR, row.image_name)
        if not os.path.exists(img_path):
            continue

        gt = int(row['count'])
        pred = count_persons_yolov5(img_path)

        gt_counts.append(gt)
        pred_counts.append(pred)

gt_counts = np.array(gt_counts)
pred_counts = np.array(pred_counts)

mse = mean_squared_error(gt_counts, pred_counts)
mae = mean_absolute_error(gt_counts, pred_counts)
pearson_r, _ = stats.pearsonr(gt_counts, pred_counts)

within_1 = np.mean(np.abs(gt_counts - pred_counts) <= 1) * 100

print(f"Frames evaluated: {len(gt_counts)}")
print(f"MSE (count):       {mse:.2f}")
print(f"MAE (count):       {mae:.2f}")
print(f"Pearson r:         {pearson_r:.3f}")
print(f"Accuracy |error|<=1: {within_1:.1f}%")


In [ ]:
# Save the entire model (architecture + weights + your tuned params)
torch.save(model, "yolov5m_people_crowd.pt")

print("✅ YOLOv5 crowd model saved successfully!")

In [ ]:
print("GT length:", len(gt_counts))
print("Pred length:", len(pred_counts))


In [ ]:
if len(gt_counts) == 0 or len(pred_counts) == 0:
    print("ERROR: gt_counts or pred_counts is empty.")
else:
    plt.figure(figsize=(6,6))

    plt.scatter(gt_counts, pred_counts, s=25, alpha=0.7)

    min_v = min(gt_counts.min(), pred_counts.min())
    max_v = max(gt_counts.max(), pred_counts.max())

    # Perfect prediction reference line
    plt.plot([min_v, max_v], [min_v, max_v], 'r--', linewidth=2)

    plt.xlabel("Ground Truth Count")
    plt.ylabel("YOLOv5n Predicted Count")
    plt.title("Occupancy Counting Performance")
    plt.grid(True)
    plt.show()

In [ ]:
import cv2

def show_yolo_detection(frame_idx=500):
    row = df.iloc[frame_idx]
    img_path = os.path.join(FRAMES_DIR, row.image_name)

    gt_count = int(row["count"])

    # YOLO prediction
    results = model(img_path, size=640)
    pred_count = len(results.xyxy[0])

    # Draw boxes
    results.render()
    img_bgr = results.imgs[0]
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(7,5))
    plt.imshow(img_rgb)
    plt.axis("off")
    plt.title(f"YOLOv5n Person Detection | GT: {gt_count}  Pred: {pred_count}")
    plt.show()

    print("Frame ID      :", row.frame_id)
    print("Ground Truth  :", gt_count, "people")
    print("YOLO Predicted:", pred_count, "people")

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import scipy.stats as stats

mae = mean_absolute_error(gt_counts, pred_counts)
mse = mean_squared_error(gt_counts, pred_counts)
pearson_r, _ = stats.pearsonr(gt_counts, pred_counts)

within_1 = np.mean(np.abs(gt_counts - pred_counts) <= 1) * 100
within_2 = np.mean(np.abs(gt_counts - pred_counts) <= 2) * 100

print("===== OCCUPANCY COUNTING ACCURACY REPORT =====")
print(f"MAE (Mean Absolute Error): {mae:.2f} people")
print(f"MSE (Mean Squared Error): {mse:.2f}")
print(f"Pearson Correlation (r):  {pearson_r:.3f}")
print(f"Accuracy within ±1:      {within_1:.2f}%")
print(f"Accuracy within ±2:      {within_2:.2f}%")


In [ ]:
# Test on a single sample image (GT vs Predicted)

import matplotlib.pyplot as plt
import cv2

def test_single_frame(frame_idx):
    """
    frame_idx: 0-based index into df (0..len(df)-1)
    """

    row = df.iloc[frame_idx]
    img_path = os.path.join(FRAMES_DIR, row.image_name)

    # Ground-truth count (from mall_gt)
    gt_count = int(row["count"])

    # YOLOv5 prediction
    results = model(img_path, size=640)
    pred_count = len(results.xyxy[0])   # number of detections (persons)

    # Render YOLO boxes on image
    annotated_imgs = results.render()
    img_bgr = annotated_imgs[0]
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # Show image
    plt.figure(figsize=(7,5))
    plt.imshow(img_rgb)
    plt.axis("off")
    plt.title(f"Frame {row.frame_id}  |  GT: {gt_count}  |  Predicted: {pred_count}")
    plt.show()

    print(f"Frame ID      : {row.frame_id}")
    print(f"Ground truth  : {gt_count} people")
    print(f"YOLO predicted: {pred_count} people")

# choose any index and test
test_single_frame(470)


In [ ]:
# 9. Optional: Kalman filter smoothing over time

class KalmanCountFilter:
    def __init__(self, process_var=0.5, meas_var=4.0, initial_count=0.0):
        self.x = initial_count
        self.P = 1.0
        self.Q = process_var
        self.R = meas_var

    def update(self, z):
        # predict
        x_pred = self.x
        P_pred = self.P + self.Q
        # update
        K = P_pred / (P_pred + self.R)
        self.x = x_pred + K * (z - x_pred)
        self.P = (1 - K) * P_pred
        return self.x

kf = KalmanCountFilter(process_var=0.5, meas_var=4.0, initial_count=0.0)

smooth_counts = []
for c in pred_counts:
    smooth_counts.append(kf.update(c))

# Plot raw vs smoothed counts
plt.figure(figsize=(12,4))
plt.plot(gt_counts, label="GT count", alpha=0.7)
plt.plot(pred_counts, label="Raw YOLO count", alpha=0.6)
plt.plot(smooth_counts, label="Kalman-smoothed", linewidth=2)
plt.xlabel("Frame index (subset)")
plt.ylabel("Occupancy")
plt.title("Occupancy Over Time")
plt.legend()
plt.show()